# 手写字体识别

## 数据预处理

[python读取npz文件](https://blog.csdn.net/weixin_42499236/article/details/84027378)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
path = r'D:\JWE\Files\Courses\数据挖掘\work\第十、十一章\data'
file = r'mnist.npz'

data = np.load(path + '\\' + file)

In [5]:
data.files

['x_test', 'x_train', 'y_train', 'y_test']

In [9]:
data['x_train'].shape
data['x_test'].shape
data['y_train'].shape
data['y_test'].shape

(60000, 28, 28)

(10000, 28, 28)

(60000,)

(10000,)

## CNN 

[keras搭建CNN](https://blog.csdn.net/qq_35082030/article/details/79144374)

In [35]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD

In [39]:
X_train = data['x_train']
Y_train = data['y_train']
X_test = data['x_test']
Y_test = data['y_test']

X_train = X_train.reshape(X_train.shape[0],28,28,1)/255
X_test = X_test.reshape(X_test.shape[0],28,28,1)/255
Y_train = np_utils.to_categorical(Y_train,num_classes=10)
Y_test = np_utils.to_categorical(Y_test,num_classes=10)

In [41]:
def CNN_create(x_train=X_train,y_train=Y_train,divide=0.2,batch_size=30,epochs=200,drop_rate=0.5):
    CNN = Sequential()
    # 一次卷积
    CNN.add(Conv2D(25,(3,3),activation='relu',input_shape=(28,28,1))) # 卷积层
    CNN.add(MaxPooling2D(pool_size=(2,2))) # 池化层
#     CNN.add(Dropout(0.5)) # 遗忘层
    # 二次卷积
    CNN.add(Conv2D(50,(3,3), activation='relu'))
    CNN.add(MaxPooling2D(pool_size=(2,2)))
#     CNN.add(Dropout(0.5))
    CNN.add(Flatten()) # 压平层
    CNN.add(Dense(100,activation='relu')) # 全连接层
#     CNN.add(Dropout(0.5)) # 遗忘层
    CNN.add(Dense(10,activation='softmax')) # 输出层
    CNN.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) # 'binary_crossentropy' 二分类
    CNN.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_split=divide)
    return CNN

In [45]:
CNN = CNN_create(x_train=X_train,y_train=Y_train,batch_size=4000,epochs=5)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 22s 449us/step - loss: 1.9794 - accuracy: 0.5521 - val_loss: 1.3520 - val_accuracy: 0.7467
Epoch 2/5
48000/48000 [==============================] - 22s 462us/step - loss: 0.8795 - accuracy: 0.7862 - val_loss: 0.4658 - val_accuracy: 0.8677
Epoch 3/5
48000/48000 [==============================] - 23s 484us/step - loss: 0.4080 - accuracy: 0.8792 - val_loss: 0.2976 - val_accuracy: 0.9138
Epoch 4/5
48000/48000 [==============================] - 22s 451us/step - loss: 0.2925 - accuracy: 0.9133 - val_loss: 0.2351 - val_accuracy: 0.9329
Epoch 5/5
48000/48000 [==============================] - 22s 454us/step - loss: 0.2289 - accuracy: 0.9313 - val_loss: 0.1870 - val_accuracy: 0.9459


In [46]:
CNN.evaluate(X_train,Y_train)

60000/60000 [==============================] - 12s 195us/step


[0.19714351520091294, 0.9409666657447815]

In [59]:
output = CNN.predict(X_test)
result = [np.argmax(each) for each in output]

In [62]:
CNN_output = pd.DataFrame({'id':[i+1 for i in range(len(result))],'label':result})
CNN_output.to_csv(r'D:\JWE\Files\Courses\数据挖掘\work\第十、十一章\CNN_output.csv',index=False)